In [1]:
from chat_with_pandas_csv import chat_with_pandas_dataframe

c:\Users\91790\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
path = 'hotel_bookings_base.csv'

In [3]:
query = "Which hotel type is sold the most"

In [4]:
res = chat_with_pandas_dataframe(query, path)

---

In [11]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///hotel_bookings.db")
print(db.dialect)
print(db.get_usable_table_names())

sqlite
[]


In [2]:
from typing_extensions import TypedDict 


class State(TypedDict): 
    question: str 
    query: str 
    result: str
    answer: str 

In [3]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [4]:
from langchain import hub

query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

assert len(query_prompt_template.messages) == 1
query_prompt_template.messages[0].pretty_print()

c:\Users\91790\anaconda3\Lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================ System Message ================================

Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{table_info}

Question: {input}


In [5]:
from typing_extensions import Annotated


class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

In [6]:
query = write_query({"question": "What is the total revenue of the year 2017"})
print(query)

{'query': 'SELECT SUM(revenue) AS total_revenue \nFROM sales \nWHERE year = 2017;'}


In [7]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool

def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}

In [8]:
execute_query(query)

{'result': 'Error: (sqlite3.OperationalError) no such table: sales\n[SQL: SELECT SUM(revenue) AS total_revenue \nFROM sales \nWHERE year = 2017;]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'}